In [1]:
from pathlib import Path
import sys
import os

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2



# Load sample GTFS feed

In [2]:
path = mm.DATA_DIR/'auckland_gtfs_sample.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Go West, AT Metro, Fullers Ferries, Pavlovich..."
1,timezone,Pacific/Auckland
2,start_date,20171012
3,end_date,20171220
4,num_routes,15
5,num_trips,4675
6,num_stops,1022
7,num_shapes,195
8,sample_date,20171019
9,num_routes_active_on_sample_date,15


# Create sample points

In [109]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[-1].values

%time spoints_1 = mm.sample_trip_points(feed, [trip_id])
%time spoints_2 = mm.sample_trip_points(feed, [trip_id], point_dist=0.1)
%time spoints_3 = mm.sample_trip_points(feed, [trip_id], num_points=100)

print('num stop patterns = ', len(spoints_1))
pattern = list(spoints_1.keys())[-1]


CPU times: user 440 ms, sys: 0 ns, total: 440 ms
Wall time: 440 ms
CPU times: user 464 ms, sys: 12 ms, total: 476 ms
Wall time: 464 ms
CPU times: user 468 ms, sys: 12 ms, total: 480 ms
Wall time: 470 ms
num stop patterns =  1


# Compare sample points with one actual feed shape

In [110]:
l1 = sg.LineString(spoints_1[pattern])
l2 = sg.LineString(spoints_2[pattern])
l3 = sg.LineString(spoints_3[pattern])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == pattern, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

# Match one set of sample points using various matching services and compare to GTFS shape

In [97]:
# Map match with Mapzen

mpoints = mm.match_with_mapzen({pattern: spoints_2[pattern]},
  os.environ['MAPZEN_KEY'])[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [111]:
# Map match with OSRM local server 

url = 'http://127.0.0.1:5000/match/v1/car'
mpoints = mm.match_with_osrm({pattern: spoints_2[pattern]}, url=url)[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [113]:
# Map match with Mapbox, which uses OSRM

mpoints = mm.match_with_mapbox({pattern: spoints_1[pattern]},
  os.environ['MAPBOX_KEY'])[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [114]:
# Map match with Google

mpoints = mm.match_with_google({pattern: spoints_1[pattern]},
  os.environ['GOOGLE_KEY'])[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


# Match feed, but only one trip thereof

In [115]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[-1].values

%time feed2 = mm.match_feed(feed, 'mapzen', os.environ['MAPZEN_KEY'], trip_ids=[trip_id])

CPU times: user 676 ms, sys: 0 ns, total: 676 ms
Wall time: 2.65 s


In [116]:
# Spot check one trip and shape
geom_by_shape = feed.build_geometry_by_shape(shape_ids=[shape_id], use_utm=False)
geom = geom_by_shape[shape_id]
geom_by_shape2 = feed2.build_geometry_by_shape(shape_ids=[shape_id], use_utm=False)
geom2 = geom_by_shape2[shape_id]

mappy = folium.Map(location=geom.coords[0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(geom), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(geom2), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [117]:
feed2.shapes.shape_id.nunique() == feed.shapes.shape_id.nunique()

True

In [118]:
# How many match calls needed for entire feed?

mm.get_num_match_calls(feed)

50